# ShopAssistAI - with Function Calling

In [1]:
import os
os.chdir('.')
!ls

README.md
ShopAssistAI - with Function Calling.ipynb
ShopAssistAI.ipynb
laptop_data.csv
system_design.png
updated_laptop.csv


In [2]:
import pandas as pd

In [3]:
# read original laptop data csv
df = pd.read_csv('laptop_data.csv')
df

,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,Screen Resolution,OS,Laptop Weight,Special Features,Warranty,Average Battery Life,Price,Description
0,Dell,Inspiron,i5,Intel,2.4 GHz,8GB,SSD,LCD,"15.6""",Intel UHD,1920x1080,Windows 10,2.5 kg,Backlit Keyboard,1 year,6 hours,"35,000",The Dell Inspiron is a versatile laptop that c...
1,MSI,GL65,i7,Intel,2.6 GHz,16GB,HDD+SSD,IPS,"15.6""",NVIDIA GTX,1920x1080,Windows 10,2.3 kg,RGB Keyboard,2 years,4 hours,"55,000",The MSI GL65 is a high-performance laptop desi...
2,HP,EliteBook,i7,Intel,2.8 GHz,16GB,SSD,LED,"14""",Intel UHD,1920x1080,Windows 11,1.5 kg,Fingerprint Sensor,3 years,8 hours,"90,000",The HP EliteBook is a premium laptop designed ...
3,Lenovo,IdeaPad,i3,Intel,2.1 GHz,8GB,HDD,TN,"15.6""",Intel UHD,1366x768,Windows 10,2.2 kg,Dolby Audio,1 year,5 hours,"25,000",The Lenovo IdeaPad is a versatile laptop that ...
4,ASUS,ZenBook Pro,i9,Intel,3.1 GHz,64GB,SSD,OLED,"15.6""",NVIDIA RTX,3840x2160,Windows 10,1.8 kg,NanoEdge Display,2 years,7 hours,"200,000",The ASUS ZenBook Pro is a high-end laptop that...
5,Acer,Predator,i7,Intel,2.8 GHz,16GB,SSD,IPS,"17.3""",NVIDIA GTX,1920x1080,Windows 10,3.2 kg,Dual Cooling Fans,1 year,5 hours,"80,000",The Acer Predator is a powerhouse laptop desig...
6,Microsoft,Surface Laptop,i5,Intel,1.6 GHz,8GB,SSD,PixelSense,"13.5""",Intel Iris Plus,2256x1504,Windows 11,1.3 kg,Touchscreen Display,1 year,10 hours,"90,000",The Microsoft Surface Laptop is a premium devi...
7,Lenovo,ThinkPad,Ryzen 7,AMD,3.0 GHz,16GB,SSD,IPS,"14""",NVIDIA GTX,2560x1440,Linux,1.6 kg,Backlit Keyboard,3 years,6 hours,"60,000",The Lenovo ThinkPad is a powerful laptop desig...
8,HP,Pavilion,i5,Intel,2.3 GHz,12GB,HDD,LCD,"15.6""",Intel UHD,1366x768,Windows 10,2.1 kg,B&O Audio,1 year,4 hours,"30,000",The HP Pavilion is a budget-friendly laptop th...
9,ASUS,ROG Strix G,i7,Intel,2.9 GHz,16GB,SSD,IPS,"17.3""",NVIDIA RTX,1920x1080,Windows 10,2.9 kg,Aura Sync RGB Keyboard,2 years,5 hours,"85,000",The ASUS ROG Strix G is a high-performance gam...


In [4]:
# Import the libraries
import os, json, ast
import openai

In [5]:
# initialise with the key
openai.api_key = open("../openai_api_key.txt", "r").read().strip()
os.environ['OPENAI_API_KEY'] = openai.api_key

# initialize client
client = openai.OpenAI(
  api_key=openai.api_key
)

In [57]:
# Define a Chat Completions API call
def get_chat_completions(input, json_format = False):
    """
    Generate chat completions using OpenAI's GPT-3.5-turbo model.

    Parameters:
    - input: list of dictionaries representing chat messages to prompt the model.
    - json_format: boolean, specifies whether the output should be in JSON format (default: False).

    Returns:
    - output: The generated chat completion response. If json_format is True, returns a JSON object.
    """
    
    MODEL = "gpt-4o-mini"
    
    try:
        
        messages = input
        
        response = client.chat.completions.create(
            model = MODEL,
            messages = messages,
            temperature = 0,
            response_format = {"type": "json_object" if json_format else "text"}
        )
        
        if json_format is False:
            response_content = response.choices[0].message.content
        else:
            response_content = json.loads(response.choices[0].message.content)
        
        return response_content
    
    # Raise exception error
    except Exception as e:
        print(f"An error occurred with the call to LLM: {e}")
        return None

In [64]:
# given a laptop description as an input extract the json parameters from it
def product_map_layer(laptop_description):
    print("inside product map layer")
    delimiter = "#####"
    
    lap_spec = {
      "gpu intensity":"(Type of the Graphics Processor)",
      "display quality":"(Display Type, Screen Resolution, Display Size)",
      "portability":"(Laptop Weight)",
      "multitasking":"(RAM Size)",
      "processing speed":"(CPU Type, Core, Clock Speed)"
    }

    prompt=f"""
    You are an expert laptop specifications analyst. You are an expert in extracting required information from any 
    given text pertaining to laptops. You will be given a paragraph of text as input which represents information 
    about a laptop. Your task is to extract information for the keys: 
    'GPU intensity','Display quality','Portability','Multitasking','Processing speed' based on the information 
    present in the paragraph of text. The value for each key should only be 'low', 'medium' or 'high'.
    {delimiter}
    'gpu intensity' must be decided with these rules:
    - low: Integrated graphics (e.g., Intel UHD, Intel Iris Plus, AMD Radeon integrated graphics)
    - medium: Entry-level to mid-range dedicated graphics (e.g., NVIDIA GTX, AMD Radeon)
    - high: High-end dedicated graphics (e.g., NVIDIA RTX, AMD Radeon Pro, NVIDIA Quadro)
    
    'display quality' must be decided with these rules:
    - low: Resolution below Full HD (e.g., 1366x768)
    - medium: Full HD resolution (1920x1080) or higher
    - high: High-resolution display (e.g., 4K, Retina, OLED) with excellent color accuracy and features like HDR support
    
    'portability' must be decided with these rules:
    - low: Weighs more than 2.5 kg
    - medium: Weighs between 1.5 kg and 2.5 kg
    - high: Weighs less than 1.5 kg
    
    'multitasking' must be decided with these rules:
    - low: 8GB of RAM
    - medium: 12GB to 16GB of RAM
    - high: 32GB or more of RAM
    
    'processing speed' must be decided with these rules:
    - low: Processor clock speed below 2.3 GHz. Should be considered for entry-level processors like Intel Core i3, AMD Ryzen 3
    - medium: Processor clock speed between 2.3 GHz and 2.8 GHz. Should be considered for Mid-range processors like Intel Core i5, AMD Ryzen 5
    - high: Processor clock speed above 2.8 GHz. Should be considered for High-performance processors like Intel Core i7, AMD Ryzen 7 or higher
    {delimiter}
    
    {delimiter}
    Here are some example outputs for a better understanding of the task:
    input1: "The Dell Inspiron is a versatile laptop that combines powerful performance and affordability. It features an Intel Core i5 processor clocked at 2.4 GHz, ensuring smooth multitasking and efficient computing. With 8GB of RAM and an SSD, it offers quick data access and ample storage capacity. The laptop sports a vibrant 15.6" LCD display with a resolution of 1920x1080, delivering crisp visuals and immersive viewing experience. Weighing just 2.5 kg, it is highly portable, making it ideal for on-the-go usage. Additionally, it boasts an Intel UHD GPU for decent graphical performance and a backlit keyboard for enhanced typing convenience. With a one-year warranty and a battery life of up to 6 hours, the Dell Inspiron is a reliable companion for work or entertainment. All these features are packed at an affordable price of 35,000, making it an excellent choice for budget-conscious users."
    output1: {{'gpu intensity': 'medium','display quality':'medium','portability':'medium','multitasking':'high','processing speed':'medium'}}
    
    input2: "The Lenovo ThinkPad X1 Carbon is a sleek and lightweight laptop designed for professionals on the go. It is equipped with an Intel Core i7 processor running at 2.6 GHz, providing strong processing capabilities for multitasking and productivity. With 16GB of RAM and an SSD, it offers fast and efficient performance along with ample storage capacity. The laptop features a 14" IPS display with a resolution of 2560x1440, delivering sharp visuals and accurate colors. It comes with Intel UHD integrated graphics for decent graphical performance. Weighing just 1.13 kg, it is extremely lightweight and highly portable. The laptop features an IR camera for face unlock, providing convenient and secure login options. With a three-year warranty and an impressive battery life of up to 12 hours, the Lenovo ThinkPad X1 Carbon ensures reliability and long-lasting productivity. Priced at 130,000, it offers top-notch performance and portability for professionals."
    output2: {{'gpu intensity': 'medium', 'display quality': 'high', 'portability': 'high', 'multitasking':'high', 'processing speed':'high'}}
    
    input3: "The Apple MacBook Pro is a high-end laptop that combines top-tier performance with a stunning display. It is equipped with an Intel Core i9 processor running at 2.9 GHz, providing exceptional processing power for demanding tasks and content creation. With 32GB of RAM and an SSD, it offers seamless multitasking and fast storage access for large projects. The laptop features a 16" Retina display with a resolution of 3072x1920, delivering breathtaking visuals and precise color reproduction. It comes with an AMD Radeon graphics card, ensuring smooth graphics performance for professional applications. Weighing 2.02 kg, it is relatively lightweight for its size. The laptop features a True Tone display, adjusting the color temperature to match the ambient lighting for a more natural viewing experience. With a three-year warranty and a battery life of up to 10 hours, the Apple MacBook Pro offers reliability and endurance for professionals. Priced at 280,000, it caters to users who require uncompromising performance and a superior display for their demanding workloads."
    output3: {{'gpu intensity': 'medium', 'display quality': 'high', 'portability': 'medium','multitasking': 'high', 'processing speed': 'high'}}
    {delimiter}
    
    The output should be in the form of a json with the format as {lap_spec}. No other information needs to be present 
    in the output other than the laptop specifications json.
    """

    input = f"""Follow the prompt instructions step-by-step and output the dictionary in JSON format for the 
    following laptop {laptop_description}."""

    messages=[{"role": "system", "content":prompt},{"role": "user","content":input}]

    response = get_chat_completions(messages, json_format=True)

    return response

In [65]:
# testing product mapping function
laptop_description_1 = f"""
The Dell Inspiron is a versatile laptop that combines powerful performance and affordability.
It features an Intel Core i5 processor clocked at 2.4 GHz, ensuring smooth multitasking and efficient computing.
With 8GB of RAM and an SSD, it offers quick data access and ample storage capacity.
The laptop sports a vibrant 15.6" LCD display with a resolution of 1920x1080, delivering crisp visuals and immersive viewing experience.
Weighing just 2.5 kg, it is highly portable, making it ideal for on-the-go usage.
Additionally, it boasts an Intel UHD GPU for decent graphical performance and a backlit keyboard for enhanced typing convenience.
With a one-year warranty and a battery life of up to 6 hours, the Dell Inspiron is a reliable companion for work or entertainment.
All these features are packed at an affordable price of 35,000, making it an excellent choice for budget-conscious users.
"""
display(product_map_layer(laptop_description_1))

inside product map layer


{'gpu intensity': 'medium',
 'display quality': 'medium',
 'portability': 'medium',
 'multitasking': 'high',
 'processing speed': 'medium'}

In [66]:
##Run this code once to extract product info in the form of a dictionary
laptop_df= pd.read_csv('laptop_data.csv')

## Create a new column "laptop_feature" that contains the dictionary of the product features
laptop_df['laptop_feature'] = laptop_df['Description'].apply(lambda x: product_map_layer(x))

laptop_df.head()

inside product map layer
inside product map layer
inside product map layer
inside product map layer
inside product map layer
inside product map layer
inside product map layer
inside product map layer
inside product map layer
inside product map layer
inside product map layer
inside product map layer
inside product map layer
inside product map layer
inside product map layer
inside product map layer
inside product map layer
inside product map layer
inside product map layer
inside product map layer


,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,Screen Resolution,OS,Laptop Weight,Special Features,Warranty,Average Battery Life,Price,Description,laptop_feature
0,Dell,Inspiron,i5,Intel,2.4 GHz,8GB,SSD,LCD,"15.6""",Intel UHD,1920x1080,Windows 10,2.5 kg,Backlit Keyboard,1 year,6 hours,"35,000",The Dell Inspiron is a versatile laptop that c...,"{'gpu intensity': 'medium', 'display quality':..."
1,MSI,GL65,i7,Intel,2.6 GHz,16GB,HDD+SSD,IPS,"15.6""",NVIDIA GTX,1920x1080,Windows 10,2.3 kg,RGB Keyboard,2 years,4 hours,"55,000",The MSI GL65 is a high-performance laptop desi...,"{'gpu intensity': 'medium', 'display quality':..."
2,HP,EliteBook,i7,Intel,2.8 GHz,16GB,SSD,LED,"14""",Intel UHD,1920x1080,Windows 11,1.5 kg,Fingerprint Sensor,3 years,8 hours,"90,000",The HP EliteBook is a premium laptop designed ...,"{'gpu intensity': 'medium', 'display quality':..."
3,Lenovo,IdeaPad,i3,Intel,2.1 GHz,8GB,HDD,TN,"15.6""",Intel UHD,1366x768,Windows 10,2.2 kg,Dolby Audio,1 year,5 hours,"25,000",The Lenovo IdeaPad is a versatile laptop that ...,"{'gpu intensity': 'low', 'display quality': 'l..."
4,ASUS,ZenBook Pro,i9,Intel,3.1 GHz,64GB,SSD,OLED,"15.6""",NVIDIA RTX,3840x2160,Windows 10,1.8 kg,NanoEdge Display,2 years,7 hours,"200,000",The ASUS ZenBook Pro is a high-end laptop that...,"{'gpu intensity': 'high', 'display quality': '..."


In [67]:
# all the updated data is now being stored inside updated_laptop.csv
laptop_df.to_csv("updated_laptop.csv",index=False,header = True)

In [102]:
# function to initialize the conversation with the AI assistant
# this would be the system message for the api call
def initialize_conversation():
    '''
    Returns a list [{"role": "system", "content": system_message}]
    '''
    
    delimiter = "####"

    system_message = f"""
    You are an expert laptop recommendation system. You evaluate requests for laptops based on the following 
    parameters: GPU intensity, display quality, portability, multitasking, processing speed, and budget.
    You need to ask relevant questions to the user in case these points needed are not satisfied 
    as per the input query.
    
    Based on the input query given by the user, you will need to determine details for the following keys 
    only: ('gpu intensity', 'display quality', 'portability', 'multitasking', 'processing speed', 'budget').
    The values taken in for all the keys as shown above other than budget need to be as per ('low', 'medium', 'high').
    The assignment of these values should be based on the importance given by the user for each of the parameters 
    in consideration. The value for budget needs to be filled in with the actual value as given by the user.
    
    Once you ask the relevant questions and get all the details from the user, your goal is to fetch details of the 
    top 3 laptops which match the specifications as given by the user by using function calling with 
    the compare_laptops_with_user function.
    
    {delimiter}
    Once you get the list of top 3 laptops you will need to neatly format it and show the recomendations to the 
    user in the following format:
    1. <Laptop name>: <Basic laptop specs in brief>, <price of laptop>
    2. <Laptop name>: <Basic laptop specs in brief>, <price of laptop>
    3. <Laptop name>: <Basic laptop specs in brief>, <price of laptop>
    {delimiter}
    
    {delimiter}
    Here are some instructions for the values assigned for the different keys. 
    If you do not follow this, you'll be heavily penalised.
    - The values for all keys, except 'Budget', should strictly be either 'low', 'medium', or 'high' based on 
    the importance of the corresponding keys, as stated by user.
    - The value for 'budget' should be a numerical value extracted from the user's response.
    - 'budget' value needs to be greater than or equal to 25000 INR. If the user says less than that, 
    please mention that there are no laptops in that range.
    - Do not randomly assign values to any of the keys. The values need to be inferred from the user's response.
    - The final goal is to display the python dictionary only once the values for all the parameters have been 
    decided. The final output after the conversation should not have any other text in it other than the python dictionary.
    - Do not explicitly ask the user to input the values as low, medium or high. These values should be inferred from 
    the response given by the user for the questions being asked.
    - At a given time, do not ask the user questions regarding more than 1 parameter. For instance, if the processing speed 
    requirement and multitasking requirement need to be decided, only ask a question regarding that. Refrain from 
    asking about both the parameters within one query to the user.
    {delimiter}
    
    Follow the steps defined below to ensure that you get the required information from the user for all the 
    parameters:
    {delimiter}
    Step 1: Greet the user with a short message asking about the type of laptop they need or the purpose for 
    which they need it.
    {delimiter}
    Step 2: Based on the input given by the user, try to infer information for the keys as discussed above and 
    try to fill up the python dictionary only for the keys that are relevant.
    {delimiter}
    Step 3: In case the response given by the user doesn't satisfactorily cover all the keys of the python dictionary, 
    then based on the keys that are missed out, ask the user some basic questions to enable them to fill in information 
    that can help you fill the values for the remaining keys in the dictionary.
    {delimiter}
    Step 4: Repeat steps 2 and 3 until the python dictionary has all the necessary values filled up with a good level 
    of confidence. Remember to fill in the values only based on the user's input and do not fill in the values by 
    yourself. Remember to ask simple questions that cover only one parameter at a time. Do not ask questions regarding 
    more than one parameter at a time.
    {delimiter}
    Follow the above chain of thoughts.
    
    {delimiter}
    A sample conversation with the user can be as follows:
    User: "I am a gamer. What type of laptop would be good for me?",
    Assistant: "Since you are a gamer is it safe to say that you need a computer that can handle high quality 
    graphic content? Further, you would also be requiring a good quality display to complement your gaming needs.",
    User: "Yes, that is correct. I generally play games like GTA 5, Cyberpunk 2077, Call of Duty. However, it is 
    not important for me to play these games on the highest settings possible. I would even enjoy them with a 
    medium to high graphics setting.",
    Assistant: "Thank you for that information, that certainly helps me understand your needs better. Can you give 
    me some information on whether you would be having a need to carry your laptop around to multiple places or 
    would you use it primarily at one location?",
    User: "I don't need to move my laptop around much. I would be mostly playing with the laptop at home only.",
    Assistant: "Would you be doing a lot of tasks on your computer at the same time? Or is it mostly only going 
    to be for gaming? This would help me understand your multitasking needs as well as help me to understand the 
    kind of processor that you require",
    User: "I generally wouldn't use my laptop for a lot of other purposes than gaming. There is hardly a chance 
    for me to do multiple things at the same time.",
    Assistant: "That's great information. Finally, can you please give me some information on the budget that is 
    on your mind for the laptop?",
    User: "Below 70000 INR"
    
    {delimiter} 
    Here is another sample conversation between the user and assistant:
    User: "Hi, I am an editor."
    Assistant: "Great! As an editor, you likely require a laptop that can handle demanding tasks. Hence, the 
    laptop should have high multitasking capability. You would also need a high end display for better visuals 
    and editing. May I know what kind of work do you primarily focus on? Are you more involved in video editing, 
    photo editing, or both? Understanding the specific type of editing work will help me tailor my recommendations 
    accordingly. Let me know if my understanding is correct until now."
    User: "I primarily work with After Effects."
    Assistant: "Thank you for providing that information. Working with After Effects involves working with 
    graphics, animations, and rendering, which will require high GPU. Do you work with high-resolution media 
    files, such as 4K videos or RAW photos? Understanding your file sizes will help determine the storage capacity 
    and processing power needed."
    User: "Yes, sometimes I work with 4K videos as well."
    Assistant: "Thank you for the information. Processing 4K vidoes will require a good processor and high GPU. 
    I think we have already determined earlier that you need a high GPU. To ensure I have a complete understanding 
    of your needs, I have one more question: Are you frequently on the go and require a laptop that is lightweight 
    and easy to carry, or do you primarily work from a stationary location?"
    User: "Yes, sometimes I travel but do not carry my laptop."
    Assistant:"Could you kindly let me know your budget for the laptop? This will help me find options that fit 
    within your price range while meeting the specified requirements."
    User: "my max budget is 1.5lakh inr"
    
    {delimiter} 
    Here is another sample conversation between the user and assistant:
    User: "i am a basic business user"
    Assistant: "As a basic business user, it's important to have a reliable laptop for your day-to-day tasks. 
    Based on your usage as a basic business user, do you prefer a laptop with a good quality display for 
    presentations?"
    User: "yes"
    Assistant: "Great! Do you prefer a laptop that is easy to carry around or do you primarily use it at a fixed 
    location?"
    User: "i need it to be easy to carry around"
    Assistant: "Thank you for specifying that. Do you often have multiple tasks running on your laptop 
    simultaneously or primarily focus on one task at a time?"
    User: "i ususally have multiple tasks running"
    Assistant: "Thank you for sharing that. Do you prefer a laptop with fast processing speed for quick 
    performance?"
    User: "yes i need it to be very quick"
    Assistant: "Appreciate that input. Finally, what is your budget range for the laptop you are looking to 
    purchase?"
    User: "i would like if it is below 1 lakh"
    """
    
    conversation = [{"role": "system", "content": system_message}]

    return conversation

In [99]:
# specifying this function for the function call to take place and get information regarding the 
# laptops information based on the users request

def compare_laptops_with_user(user_req_string):
    # read the updated laptop csv file with the added parameter json information
    laptop_df = pd.read_csv('updated_laptop.csv')

    # understand budget from the users requirement input dict
    user_requirements = user_req_string
    user_budget = int(user_requirements.get('budget', '0'))
    
    # make a copy of the df and format the prices to a numeric value
    laptops_list = laptop_df.copy()
    laptops_list['Price'] = laptops_list['Price'].str.replace(',','').astype(int)
    
    # get list of laptops lesser than or equal to users budget
    laptops_list = laptops_list[laptops_list['Price'] <= user_budget].copy()
    
    score_map = {
        'low': 0,
        'medium': 1,
        'high': 2
    }
    
    # create 'Score' column in the df and initialize to 0
    laptops_list['Score'] = 0
    
    # generate scores for each laptop
    for index, row in laptops_list.iterrows():
        # read the laptop_feature string which contains the json pertaining to the laptop and convert it to a dict
        laptop_feature_str = row['laptop_feature']
        laptop_dict = ast.literal_eval(laptop_feature_str)
        
        # converting keys to lowercase
        laptop_dict = {k.lower(): v for k,v in laptop_dict.items()}
        
        score = 0

        for key, user_value in user_requirements.items():
            if key.lower() == 'budget':
                continue
            laptop_value = laptop_dict.get(key.lower(), None)
            if laptop_value is not None:
                laptop_mapping = score_map.get(laptop_value.lower(), -1)
            else:
                laptop_mapping = -1
            user_mapping = score_map.get(user_value.lower(), -1)
            if laptop_mapping >= user_mapping:
                # if the laptop value is greater than or equal to the user value the score is incremented by 1
                score += 1

        laptops_list.loc[index, 'Score'] = score
        
    # sort the laptops by score in descending order and return the top 5 products
    top_laptops = laptops_list.drop('laptop_feature', axis=1)
    top_laptops = top_laptops.sort_values('Score', ascending=False).head(3)

    return top_laptops.to_json(orient='records')

In [100]:
def get_chat_completions_function_calling(input):
    
    model = 'gpt-4o-mini'
    
    # describing the function call parameters
    tools = [
        {
            "type": "function",
            "function": {
                "name": "compare_laptops_with_user",
                "description": "Get the top 3 laptops for the user from the catalogue available based on parameters like GPU intensity, display quality, portability, multitasking, processing speed, and budget",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "gpu intensity": {
                            "type": "string",
                            "description": "The GPU intensity requirement of the user specified as low, medium or high"
                        },
                        "display quality": {
                            "type": "string",
                            "description": "The Display Quality requirement of the user specified as low, medium or high"
                        },
                        "portability": {
                            "type": "string",
                            "description": "The Portability requirement of the user specified as low, medium or high"
                        },
                        "multitasking": {
                            "type": "string",
                            "description": "The Multitasking requirement of the user specified as low, medium or high"
                        },
                        "processing speed": {
                            "type": "string",
                            "description": "The Processing speed requirement of the user specified as low, medium or high"
                        },
                        "budget": {
                            "type": "integer",
                            "description": "The maximum budget of the user"
                        },
                    },
                    "required": [
                        "GPU intensity",
                        "Display quality",
                        "Portability",
                        "Multitasking",
                        "Processing speed",
                        "Budget"
                    ]
                }
            }
        }
    ]

    
    try:
        messages = input
        
        response = client.chat.completions.create(
            model = model,
            messages = messages,
            temperature = 0,
            tools = tools,
            tool_choice = 'auto'
        )
        
        # check if the model wanted to call a function
        tool_calls = response.choices[0].message.tool_calls
        
        # call the function
        if tool_calls:
            available_functions = {
                "compare_laptops_with_user": compare_laptops_with_user,
            }
            
            # append response given by gpt to input messages list
            messages.append(response.choices[0].message)
            
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_to_call = available_functions[function_name]
                function_args = json.loads(tool_call.function.arguments)
                function_response = function_to_call(function_args)
                
                function_call_response_dict = {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
                
                # append response_messages to the original input messages
                messages.append(function_call_response_dict)
                
                # make a second call to the model
                second_response = client.chat.completions.create(
                    model = model,
                    messages = messages,
                    temperature = 0
                )
                
                second_response_message = [{"role": "assistant", "content": second_response.choices[0].message.content}]
                return second_response_message
        else:
            response_message = [{"role": "assistant", "content": response.choices[0].message.content}]
            return response_message
    
    # Raise exception error
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [101]:
# test the conversation

conversation = initialize_conversation()
cnt = 0

print("Assistant:")
print("Please tell me how I can assist you with your laptop preferences?\n")

while cnt < 20:
    print("User: ")
    user_input = input()
    
    if user_input.lower() == 'exit':
        break
    
    input_add = conversation.append({"role": "user", "content": user_input})
    response = get_chat_completions_function_calling(conversation)

    print("\nAssistant:\n", response[0]['content'], '\n')
    
    conversation = conversation + response

Assistant:
Please tell me how I can assist you with your laptop preferences?

User: 
i need a laptop for basic home usage

Assistant:
 For basic home usage, it's important to have a reliable laptop that can handle everyday tasks. Could you please let me know if you have any specific preferences regarding the display quality? For example, do you prefer a good quality display for watching videos or working on documents? 

User: 
i want a decent display

Assistant:
 Thank you for that information! A decent display would indicate a medium requirement for display quality. 

Next, do you need a laptop that is easy to carry around, or will you primarily be using it at a fixed location? 

User: 
it should be easy to carry around

Assistant:
 Great! Since you prefer a laptop that is easy to carry around, I will set the portability requirement to high.

Now, do you usually have multiple tasks running on your laptop simultaneously, or do you primarily focus on one task at a time? This will help m